In [1]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
import ipyvuetify as v

In [2]:
#TODO
#undo button
#flexible player numbers
#show when all x's


# player1 = v.Select(label='Player1',items=['DJ', 'DY', 'HW', 'DH'],v_model=None)
# player2 = v.Select(label='Player2',items=['DJ', 'DY', 'HW', 'DH'],v_model=None)
# player3 = v.Select(label='Player3',items=['DJ', 'DY', 'HW', 'DH'],v_model=None)
# player4 = v.Select(label='Player4',items=['DJ', 'DY', 'HW', 'DH'],v_model=None)
# players = widgets.VBox([player1, player2, player3, player4])
# players_out = widgets.Output()
# with players_out:
#     display(players)

In [3]:
players_list = ['Player1','Player2','Player3','Player4']
# players_list = [player1.v_model,player2.v_model,player3.v_model,player4.v_model]
suspects_list = ['Mustard','Plum','Green','Peacock','Scarlet','White']
weapons_list = ['Knife','Candlestick','Pistol','Rope','Pipe','Wrench']
rooms_list = ['Hall','Lounge','Dining Room','Kitchen','Ball Room','Conservatory','Billiard Room','Library','Study']


In [4]:
suspects_df = pd.DataFrame(columns=['Suspect']+players_list)
suspects_df['Suspect'] = suspects_list
suspects_df = suspects_df.fillna('-')

In [5]:
weapons_df = pd.DataFrame(columns=['Weapon']+players_list)
weapons_df['Weapon'] = weapons_list
weapons_df = weapons_df.fillna('-')

In [6]:
rooms_df = pd.DataFrame(columns=['Room']+players_list)
rooms_df['Room'] = rooms_list
rooms_df = rooms_df.fillna('-')

In [7]:
suspects_start = v.Select(label='Suspects',items=suspects_list,multiple=True,class_='mx-2',v_model = None)
weapons_start = v.Select(label='Weapons',items=weapons_list,multiple=True,class_='mx-2',v_model = None)
rooms_start = v.Select(label='Rooms',items=rooms_list,multiple=True,class_='mx-2',v_model = None)

In [8]:
guesser = v.Select(label='Guesser',items=players_list,class_='mx-2',v_model=None)
answerer = v.Select(label='Answered by',items=players_list+['None'],class_='mx-2',v_model=None)
suspects = v.Select(label='Suspect',items=suspects_list,class_='mx-2',v_model=None)
weapons = v.Select(label='Weapon',items=weapons_list,class_='mx-2',v_model=None)
rooms = v.Select(label='Room',items=rooms_list,class_='mx-2',v_model=None)

player1_out = widgets.Output()

counter = 0
def if_player1(change):
    global card_shown
    with player1_out:
        if guesser.v_model == 'Player1':
            player1_out.clear_output()
            card_shown = v.Select(label='Card Shown',items=[suspects.v_model,weapons.v_model,rooms.v_model],class_='mx-2',v_model=None)
            display(card_shown)
        else:
            player1_out.clear_output()
guesser.observe(if_player1,names='v_model')
suspects.observe(if_player1,names='v_model')
weapons.observe(if_player1,names='v_model')
rooms.observe(if_player1,names='v_model')

disp = widgets.VBox([guesser,widgets.HBox([suspects,weapons,rooms]),answerer,player1_out])

In [9]:
start_out = widgets.Output()
start_btn = v.Btn(children=['Confirm'],style_='width: 150px',color='primary')
out = widgets.Output()

with start_out:
    display(widgets.HBox([suspects_start,weapons_start,rooms_start]),start_btn)


def start_btn_fn(widget,event,data):
    with out:
        start_out.clear_output()
        out.clear_output()
        display(widgets.VBox([disp,confirm_btn]))
        start_cards_list = [suspects_start.v_model,weapons_start.v_model,rooms_start.v_model]
        try:
            suspects_ind = np.where(suspects_df['Suspect'].isin(start_cards_list[0]))[0]
            suspects_df.iloc[suspects_ind,1:]=['O','X','X','X']
            suspects_ind_not = np.where(~suspects_df['Suspect'].isin(start_cards_list[0]))[0]
            suspects_df.iloc[suspects_ind_not,1]='X'
        except TypeError:
            suspects_ind = None

        try:
            weapons_ind = np.where(weapons_df['Weapon'].isin(start_cards_list[1]))[0]
            weapons_df.iloc[weapons_ind,1:]=['O','X','X','X']
            weapons_ind_not = np.where(~weapons_df['Weapon'].isin(start_cards_list[1]))[0]
            weapons_df.iloc[weapons_ind_not,1]='X'
        except TypeError:
            weapons_ind = None

        try:
            rooms_ind = np.where(rooms_df['Room'].isin(start_cards_list[2]))[0]
            rooms_df.iloc[rooms_ind,1:]=['O','X','X','X']
            rooms_ind_not = np.where(~rooms_df['Room'].isin(start_cards_list[2]))[0]
            rooms_df.iloc[rooms_ind_not,1]='X'
        except TypeError:
            rooms_ind = None

        display(suspects_df,weapons_df,rooms_df,new_game_btn)

start_btn.on_event('click', start_btn_fn)

In [14]:
column_headers = ['Guesser','Suspect','Weapon','Room','Answered by','Card Shown']

confirm_btn = v.Btn(children=['Confirm'],style_='width: 150px',color='primary')
init_log_out = widgets.Output()
log_out = widgets.Output()
log_list = []

log_df = pd.DataFrame(columns=column_headers)
with init_log_out:
    display(log_df)

log_row_num = -1


suspects_df_list = []
weapons_df_list = []
rooms_df_list = []
def log_confirm(widget,event,data):
    global log_row_num, suspects_df, weapons_df, rooms_df, log_df, suspects_df_list,weapons_df_list,rooms_df_list
    with log_out:
        init_log_out.clear_output()
        log_out.clear_output()
        
        suspects_df_list.append(suspects_df.copy())
        weapons_df_list.append(weapons_df.copy())
        rooms_df_list.append(rooms_df.copy())
        
        if guesser.v_model == 'Player1':
            card_shown_log = card_shown.v_model
        else:
            card_shown_log = 'None'
        log_row_num +=1
        log_df.loc[log_row_num+1]=[guesser.v_model,suspects.v_model,weapons.v_model,rooms.v_model,answerer.v_model,card_shown_log]
        
        guesser_loc = np.where(np.array(players_list)==guesser.v_model)[0][0]
        rotation_list = players_list[guesser_loc+1:]+players_list[:guesser_loc]
        rotation_list.reverse()
        rotation_list.append('None')
        try:
            answerer_loc = np.where(np.array(rotation_list) == answerer.v_model)[0][0]
        except IndexError:
            print("The Guesser and the Answerer cannot be the same player.")
            log_row_num -= 1
            log_df = log_df[:-1]
            return
        
        suspects_ind = np.where(suspects_df['Suspect']==suspects.v_model)[0][0]
        weapons_ind = np.where(weapons_df['Weapon']==weapons.v_model)[0][0]
        rooms_ind = np.where(rooms_df['Room']==rooms.v_model)[0][0]
        for player in rotation_list[:answerer_loc]:
            col_ind = np.where(np.array(players_list)==player)[0][0]+1

            suspects_df.iloc[suspects_ind,col_ind]='X'
            weapons_df.iloc[weapons_ind,col_ind]='X'
            rooms_df.iloc[rooms_ind,col_ind]='X'
        
        if answerer.v_model != 'None':
            #answerer_loc2 = np.where(np.array(players_list+['None']) == answerer.v_model)[0][0]
            answerer_loc2 = np.where(np.array(players_list) == answerer.v_model)[0][0]

            #print(players_list+['None'])
            if guesser.v_model == 'Player1':
                answer = card_shown.v_model
                if answer == card_shown.items[0]:
                    suspects_df.iloc[suspects_ind,1:]=['X','X','X','X']
                    suspects_df.iloc[suspects_ind,answerer_loc2+1]='O'
                elif answer == card_shown.items[1]:
                    weapons_df.iloc[weapons_ind,1:]=['X','X','X','X']
                    weapons_df.iloc[weapons_ind,answerer_loc2+1]='O'
                else:
                    rooms_df.iloc[rooms_ind,1:]=['X','X','X','X']
                    rooms_df.iloc[rooms_ind ,answerer_loc2+1]='O'
            else:
                if suspects_df.iloc[suspects_ind,answerer_loc2+1]!='O' and suspects_df.iloc[suspects_ind,answerer_loc2+1]!='X':
                    if weapons_df.iloc[weapons_ind,answerer_loc2+1]=='X' and rooms_df.iloc[rooms_ind ,answerer_loc2+1]=='X':
                        suspects_df.iloc[suspects_ind,1:]=['X','X','X','X']
                        suspects_df.iloc[suspects_ind,answerer_loc2+1]='O'
#                     elif weapons_df.iloc[weapons_ind,answerer_loc2+1]=='O' or rooms_df.iloc[rooms_ind,answerer_loc2+1]=='O':
#                         suspects_df.iloc[suspects_ind,answerer_loc2+1]='X'
                    else:
                        suspects_df.iloc[suspects_ind,answerer_loc2+1]=log_row_num+1
                if weapons_df.iloc[weapons_ind,answerer_loc2+1]!='O'and weapons_df.iloc[weapons_ind,answerer_loc2+1]!='X':
                    if suspects_df.iloc[suspects_ind,answerer_loc2+1]=='X' and rooms_df.iloc[rooms_ind ,answerer_loc2+1]=='X':
                        weapons_df.iloc[weapons_ind,1:]=['X','X','X','X']
                        weapons_df.iloc[weapons_ind,answerer_loc2+1]='O'
#                     elif suspects_df.iloc[suspects_ind,answerer_loc2+1]=='O' or rooms_df.iloc[rooms_ind,answerer_loc2+1]=='O':
#                         weapons_df.iloc[weapons_ind,answerer_loc2+1]='X'
                    else:
                        weapons_df.iloc[weapons_ind,answerer_loc2+1]=log_row_num+1
                if rooms_df.iloc[rooms_ind ,answerer_loc2+1]!='O' and rooms_df.iloc[rooms_ind ,answerer_loc2+1]!='X':
                    if suspects_df.iloc[suspects_ind,answerer_loc2+1]=='X' and weapons_df.iloc[weapons_ind ,answerer_loc2+1]=='X':
                        rooms_df.iloc[rooms_ind,1:]=['X','X','X','X']
                        rooms_df.iloc[rooms_ind ,answerer_loc2+1]='O'
#                     elif suspects_df.iloc[suspects_ind,answerer_loc2+1]=='O' or weapons_df.iloc[weapons_ind,answerer_loc2+1]=='O':
#                         rooms_df.iloc[rooms_ind ,answerer_loc2+1]='X'
                    else:
                        rooms_df.iloc[rooms_ind ,answerer_loc2+1]=log_row_num+1
            
            display(log_df,undo_btn)
    with out:
        out.clear_output()
        display(widgets.VBox([disp,confirm_btn]))
        display(suspects_df,weapons_df,rooms_df,new_game_btn)
        
confirm_btn.on_event('click',log_confirm)
# widgets.HBox([widgets.VBox([start_out,out]),widgets.VBox([log_out,players_out])])

SyntaxError: unexpected character after line continuation character (<ipython-input-14-50450d1a3a69>, line 89)

In [22]:
undo_btn = v.Btn(children=[v.Icon(children=['mdi-arrow-left'])],style_='width:50px')

def undo(widget,event,data):
    global log_row_num, suspects_df, weapons_df, rooms_df, log_df, suspects_df_list,weapons_df_list,rooms_df_list
    with log_out:
        log_out.clear_output()
        log_df = log_df[:-1]
        log_row_num = max(log_row_num-1,-1)
        display(log_df,undo_btn)
    with out:
        out.clear_output()

        try:
            suspects_df = suspects_df_list[-1]
            weapons_df = weapons_df_list[-1]
            rooms_df = rooms_df_list[-1]
        except:
            pass
        
        suspects_df_list = suspects_df_list[:-1]
        weapons_df_list = weapons_df_list[:-1]
        rooms_df_list = rooms_df_list[:-1]
        
        display(widgets.VBox([disp,confirm_btn]))
        display(suspects_df,weapons_df,rooms_df,new_game_btn)
undo_btn.on_event('click',undo)

Btn(children=[Icon(children=['mdi-arrow-left'])], style_='width:50px')

In [ ]:
new_game_btn = v.Btn(children=['New Game'],style_='width:150px')

def new_game(widget,event,data):
    #out.clear_output()
    global log_df,suspects_df, weapons_df, rooms_df, log_row_num
    log_df = pd.DataFrame(columns=column_headers)
    
    suspects_df = pd.DataFrame(columns=['Suspect']+players_list)
    suspects_df['Suspect'] = suspects_list
    suspects_df = suspects_df.fillna('-')
    
    weapons_df = pd.DataFrame(columns=['Weapon']+players_list)
    weapons_df['Weapon'] = weapons_list
    weapons_df = weapons_df.fillna('-')
    
    rooms_df = pd.DataFrame(columns=['Room']+players_list)
    rooms_df['Room'] = rooms_list
    rooms_df = rooms_df.fillna('-')
    
    log_row_num = -1
    
    with log_out:
        log_out.clear_output()
        #display(log_df)
    with out:
        out.clear_output()
    
    with start_out:
        display(widgets.HBox([suspects_start,weapons_start,rooms_start]),start_btn)
    display(widgets.HBox([widgets.VBox([start_out,out]),log_out]))
    display(widgets.HBox([suspects_start,weapons_start,rooms_start]),start_btn)
    
new_game_btn.on_event('click',new_game)
widgets.HBox([widgets.VBox([start_out,out]),log_out])

In [ ]:
# def highlight(s):
#     for i in range(len(s)):
#         if all(s.iloc[i,1:]=='X'):
#             return ['background-color: yellow']*5
# #     else:
# #         return ['background-color: white']*5

In [ ]:
# suspects_df.iloc[0,1:]=='-'

In [ ]:
# len(rooms_df)

IndexError: list index out of range